In [ ]:
#standard libraries
import csv
import os
import time

#custom libraries
import tensorflow as tf
import pandas as pd

In [ ]:
def read_file_format(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, value = reader.read(filename_queue)

    record_defaults = [tf.constant([], dtype=tf.int32), tf.constant([], dtype=tf.int32)]
    _, col2 = tf.decode_csv(value, record_defaults=record_defaults)
    
    example = tf.stack([col2])
    return example

In [ ]:
def input_pipeline(filenames, batch_size = 3, num_epochs = None, evaluation = False):   
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=False)

    example = read_file_format(filename_queue)
        
    min_after_dequeue = 10
    capacity = min_after_dequeue + 3 * batch_size
    example_batch = tf.train.batch(
        [example], batch_size=batch_size, capacity=capacity
    )    
    return example_batch

In [ ]:
def _activation_summary(x):
    tensor_name = x.name
    tensor_name = tensor_name.replace(':', '_')
    tensor_name = tensor_name.replace('(', '_')
    tensor_name = tensor_name.replace(')', '_')
    tensor_name = tensor_name.replace(' ', '_')

    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

In [ ]:
def make_prediction(x):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell]*num_layers)
        
    initial_state = lstm.zero_state(batch_size, tf.float32)

    with tf.variable_scope('lstm'):
        W = tf.get_variable('W', [rnn_size, vocab_size], tf.float32, tf.random_normal_initializer())
        b = tf.get_variable('b', [vocab_size], tf.float32, tf.constant_initializer(0.0))

        # Define Embedding
        embedding_mat = tf.get_variable('embedding', [vocab_size, rnn_size],
                                        tf.float32, tf.random_normal_initializer())
        embedding_output = tf.nn.embedding_lookup(embedding_mat, x)
        
        rnn_inputs = tf.split(axis=1, num_or_size_splits=seq_length, value=embedding_output)
        rnn_inputs = [tf.squeeze(x, [1]) for x in rnn_inputs]
        
    outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(rnn_inputs,
                                                                initial_state,
                                                                lstm)
    output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
    
    logits = tf.matmul(output, W) + b
    softmax_y = tf.nn.softmax(logits)
    
    return softmax_y

In [ ]:
# If we are inferring (generating text), we add a 'loop' function
# Define how to get the i+1 th input from the i th output
def inference_loop(prev, count):
    W = tf.get_variable('W')
    b = tf.get_variable('b')
    # Apply hidden layer
    prev_lin = tf.matmul(prev, W) + b
    # Get the index of the output (also don't run the gradient)
    prev_symbol = tf.stop_gradient(tf.argmax(prev_lin, 1))
    # Get embedded vector
    output = tf.nn.embedding_lookup(embedding_mat, prev_symbol)
    return output

In [ ]:
def calculate_loss(logits, labels):
    loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
        [logits],
        [labels],
        [tf.ones([batch_size * seq_length], dtype=tf.float32)])
    
    tf.add_to_collection('losses', loss)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [ ]:
def _add_loss_summaries(total_loss):
    # Compute the moving average of all individual losses and the total loss.
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])

    # Attach a scalar summary to all individual losses and the total loss; do the
    # same for the averaged version of the losses.
    for l in losses + [total_loss]:
    # Name each loss as '(raw)' and name the moving average version of the loss
    # as the original loss name.
        l_name = l.name.replace(":", "_")
        tf.summary.scalar(l_name + '_raw_', l)
        tf.summary.scalar(l_name, loss_averages.average(l))

    return loss_averages_op

In [ ]:
def train(total_loss, global_step):
 
    # Generate moving averages of all losses and associated summaries.
    loss_averages_op = _add_loss_summaries(total_loss)

    # Compute gradients.
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.AdamOptimizer(learning_rate)
        grads = opt.compute_gradients(total_loss)
        trunc_grads = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in grads]

    # Apply gradients.
    apply_gradient_op = opt.apply_gradients(trunc_grads, global_step=global_step)

    # Add histograms for trainable variables.
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)

    # Add histograms for gradients.
    for grad, var in trunc_grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)

    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(
        0.9, global_step
    )
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')

    return train_op

In [ ]:
global data_path
data_path = "data"

global vocab_file
vocab_file = "vocab1.csv"

global train_file
train_file = "train1.csv"

global model_path
model_path = 'VanillaLSTM'

In [ ]:
# Download/store Shakespeare data
full_model_dir = os.path.join(data_path, model_path)

# Make Model Directory
if not os.path.exists(full_model_dir):
    os.makedirs(full_model_dir)

In [ ]:
vocab = pd.read_csv("{0}/{1}".format(data_path, vocab_file),
                    header=None)

In [ ]:
global vocab_size
vocab_size = len(vocab)

global rnn_size
rnn_size = 256

global num_layers
num_layers = 1

global batch_size
batch_size = 16

global seq_length
seq_length = 16

global num_epochs
num_epochs = 1

global learning_rate
learning_rate = 0.0001

global momentum
momentum = 0.9

global save_every
save_every = 100

global print_every
print_every = 10

global logdir
logdir = 'TF_Logs'

In [ ]:
with tf.Graph().as_default():

    global_step = tf.Variable(0, name='global_step', trainable=False)

    x = tf.placeholder(tf.int32, shape=[batch_size, seq_length])
    y_ = tf.placeholder(tf.int32, shape=[batch_size, seq_length])

    y_hat = make_prediction(x)

    loss = calculate_loss(y_hat, y_)

    train_op = train(loss, global_step=global_step)

#     accuracy = evaluate_accuracy(y_hat, y_)

    example_feed = input_pipeline(["{0}/{1}".format(data_path, train_file)],
                                  batch_size = batch_size, num_epochs = num_epochs)
        
    with tf.Session() as sess: 
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(logdir, sess.graph)
        
        #initialize all variables
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        
        # Start populating the filename queue.
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        
        saver = tf.train.Saver(tf.global_variables())
        
        while not coord.should_stop():
            try:
                start_time = time.time()                

                example_batch = sess.run(example_feed)
                last_example = example_batch[-1]
                if global_step == 0:
                    example_batch = [[0]] + example_batch
                else:
                    example_batch = last_example + example_batch
                label_batch = example_batch[1:]
                
                result, summary =  sess.run([train_op, merged],
                                            feed_dict={x: example_batch})
                                
                writer.add_summary(summary, global_step.eval())
                
                duration = time.time() - start_time
                if global_step % print_every == 0:
                    summary_nums = (global_step, duration, loss)
                    print('Iteration: {}, Last Step Duration: {}, Loss: {:.2f}'.format(*summary_nums))
        
                # Save the model and the vocab
                if global_step % save_every == 0:
                    # Save model
                    model_file_name = os.path.join(full_model_dir, 'model')
                    saver.save(sess, model_file_name, global_step=global_step)
                    print('Model Saved To: {}'.format(model_file_name))

            except (tf.errors.OutOfRangeError, tf.errors.InvalidArgumentError) as e:
           
                print('Done training for %d epochs, %d steps.' % (num_epochs, step))
                # When done, ask the threads to stop.
                coord.request_stop()

        
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()